[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sapiosciences/sapio-py-tutorials/blob/master/5_notebook_experiment.ipynb)

In [1]:
from sapiopylib.rest.User import SapioUser
from sapiopylib.rest.DataMgmtService import DataMgmtServer

user = SapioUser(url="https://linux-vm:8443/webservice/api",
                 guid="3c232543-f407-4828-aae5-b33d4cd31fa7", account_name="sapio",
                 username="pyRestTest", password="Password1!", verify_ssl_cert=False)
elnManager = DataMgmtServer.get_eln_manager(user)

We can create a new notebook and add a few entries to it. In this simple version it has no parent and not based on any templates.


In [2]:
from sapiopylib.rest.pojo.DataRecord import DataRecord
from sapiopylib.rest.AccessionService import AccessionDataFieldCriteriaPojo
from sapiopylib.rest.pojo.eln.SapioELNEnums import ElnEntryType
from sapiopylib.rest.pojo.eln.ExperimentEntryCriteria import ElnEntryCriteria
from sapiopylib.rest.pojo.eln.ElnExperiment import InitializeNotebookExperimentPojo, ElnExperiment

#Create new ELN experiment
experiment: ElnExperiment = elnManager.create_notebook_experiment(InitializeNotebookExperimentPojo('Python Test'))

#Create a sample entry
sample_entry = elnManager.add_experiment_entry(experiment.notebook_experiment_id,
                                               ElnEntryCriteria(ElnEntryType.Table, 'Samples', "Sample", order=1))

#Accession 5 Samples
accessionManager = DataMgmtServer.get_accession_manager(user)
sample_accession_criteria = AccessionDataFieldCriteriaPojo('Sample', 'SampleId', 'PyWebDemo')
sample_accession_criteria.prefix = 'PyWebDemo-'
sample_id_list = accessionManager.accession_for_field(5, sample_accession_criteria)
dataRecordManager = DataMgmtServer.get_data_record_manager(user)
samples: list[DataRecord] = dataRecordManager.add_data_records("Sample", 5)
for sample_id, sample in zip(sample_id_list, samples):
    sample.set_field_value('SampleId', sample_id)
dataRecordManager.commit_data_records(samples)

#Add these 5 samples to the samples table
elnManager.add_records_to_table_entry(experiment.notebook_experiment_id, sample_entry.entry_id, samples)
elnManager.submit_experiment_entry(experiment.notebook_experiment_id, sample_entry.entry_id)

We can also add other entries, such as a text entry and a form entry with appropriate calls.

In [3]:
from sapiopylib.rest.pojo.datatype.FieldDefinition import VeloxStringFieldDefinition
from sapiopylib.rest.pojo.datatype.FieldDefinition import VeloxDoubleFieldDefinition
from sapiopylib.rest.pojo.eln.ExperimentEntry import ExperimentTextEntry, ExperimentFormEntry
from sapiopylib.rest.pojo.eln.SapioELNEnums import ElnBaseDataType, get_eln_text_entry_field_name

text_field: VeloxStringFieldDefinition = VeloxStringFieldDefinition(ElnBaseDataType.EXPERIMENT_DETAIL.data_type_name, 'TextOne',
                                                                  'Text 1', default_value='Text 1')
numeric_field: VeloxDoubleFieldDefinition = VeloxDoubleFieldDefinition(ElnBaseDataType.EXPERIMENT_DETAIL.data_type_name,
                                                                       "NumberOne", "Number One", default_value=1.0)
criteria: ElnEntryCriteria = ElnEntryCriteria(ElnEntryType.Form, "Data Form", ElnBaseDataType.EXPERIMENT_DETAIL.data_type_name,
                                              order=2, field_definition_list=[text_field, numeric_field],
                                              dependency_set=[sample_entry.entry_id])
form_entry: ExperimentFormEntry = elnManager.add_experiment_entry(experiment.notebook_experiment_id, criteria)

criteria = ElnEntryCriteria(ElnEntryType.Text, "Text 1", ElnBaseDataType.TEXT_ENTRY_DETAIL.name, order=3)
text_entry: ExperimentTextEntry = elnManager.add_experiment_entry(experiment.notebook_experiment_id, criteria)
#To modify text in the text entry we will need to get the record and set values on it.
text_record = dataRecordManager.query_system_for_record(text_entry.data_type_name, text_entry.record_id)
#Text entries data record always exists, and you cannot set record ID to some other ID.
text_record.set_field_value(get_eln_text_entry_field_name(), 'My New Text')
dataRecordManager.commit_data_records([text_record])